In [ ]:
import numpy as np # type: ignore
import sys
import os

from scipy.linalg import qr # type: ignore
from scipy.linalg import svd # type: ignore
import numpy as np # type: ignore
import transforms # type: ignore

import time
import random

print(sys.executable)

In [17]:
def my_matmul(A, B):
  n, m = A.shape
  _, k = B.shape
  C = np.zeros((n, k))
  for i in range(0, n):
    for j in range(0, k):
      for l in range(0, m):
        C[i][j] += A[i][l] * B[l][j]
  return C

In [18]:
def matmul(A, B):
  return A @ B

In [19]:
def hadamard_projection(A, k, p=5):
    m, n = A.shape
    power = 1
    l = k + p
    start = time.time()
    Y = A
    for i in range(0, m):
      Y[i, :] = transforms.fast_hadamard_transform(Y[i, :])
    random_cols = np.random.choice(n, l, replace = False)
    Y = Y[:, random_cols]
    return Y

In [20]:
def randomized_subspace_iteration_with_hadamar(A, k, p=3, q=3):
    _, n = A.shape

    l = k + p
    Y0 = hadamard_projection(A, k, p)

    Q0, _ = np.linalg.qr(Y0)
    Qj = Q0
    for j in range(q):
        Y_tilde_j = A.conj().T @ Qj
        Q_tilde_j, _ = np.linalg.qr(Y_tilde_j)

        Yj = A @ Q_tilde_j
        Qj, _ = np.linalg.qr(Yj)

    return Qj

In [21]:
def randomized_subspace_iteration(A, k, p, q=3, is_custom_mul=True):
    _, n = A.shape
    l = k + p
    Omega = np.random.randn(n, l)
    if is_custom_mul:
        Y0 = my_matmul(A, Omega)
    else:
        Y0 = A @ Omega

    Q0, _ = np.linalg.qr(Y0)
    Qj = Q0
    for j in range(q):
        Y_tilde_j = A.conj().T @ Qj
        Q_tilde_j, _ = np.linalg.qr(Y_tilde_j)

        Yj = A @ Q_tilde_j
        Qj, _ = np.linalg.qr(Yj)

    return Qj

In [22]:
def get_loss(A, Q):
  return np.linalg.norm(Q @ Q.T @ A - A) / np.linalg.norm(A)

In [23]:
# def check_loss_k(A, get_Q_func, k, p=3, q=3, max_loss = 1e-14):
#     Q = get_Q_func(A, k, p, q)
#     loss = get_loss(A, Q)
#     return loss < max_loss

In [24]:
# def get_rank_binary_search(A, left, right, get_Q_func, p=3, q=3, max_loss = 1e-14):
#   while right - left > 1: 
#     k = (right + left) // 2
#     if (check_loss_k(A, get_Q_func, k, p, q, max_loss)):
#       right = k
#     else:
#       left = k
#   return right

In [25]:
def get_rank(A, get_Q, p=3, q=3, max_loss = 1e-14):
  k = 1
  while True:
    Q = get_Q(A, k, p, q)
    loss = get_loss(A, Q)
    print(f'now k = {k}, now loss = {loss}')
    if loss > max_loss:
      k *= 2
    else:
      return k


In [26]:
def random_rank_k_matrix(m, n, k, random_state=None):
    B = np.random.randn(m, k)
    C = np.random.randn(k, n)
    A: np.matrix = B @ C
    return A

In [27]:
k = 63
p = 10
q = 3

In [28]:
m, n = 2**5, 2**5

In [29]:
A = random_rank_k_matrix(m, n, k)

In [ ]:
time_start = time.time()
rank = get_rank(A, randomized_subspace_iteration_with_hadamar)
time_finish = time.time()
print(rank)
print(f'time = {time_finish - time_start}')

In [ ]:
time_start = time.time()
np_rank = np.linalg.matrix_rank(A)
time_finish = time.time()
print(np_rank)
print(f'time = {time_finish - time_start}')